In [1]:
import numpy as np
import math
from QuantizedDense import Quantizeweights,bitshift,MatrixMul,MatrixBitShift
import tensorflow as tf
from keras.src import ops

In [11]:
x=str(-1)
n=x[1:]
y=int(n)
print(y)


1


In [20]:
x=tf.constant([0.0034,-0.12, 0.045, 0.2, 1, -1.05, 2.34, -0.44, 0.5])
y=tf.reduce_max(x)
c=x/y
c

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([ 0.00145299, -0.05128205,  0.01923077,  0.08547009,  0.42735043,
       -0.44871795,  1.        , -0.18803419,  0.21367522], dtype=float32)>

In [45]:
def LQ(tensor, bitwidth):
    # Create a mask for weights equal to 0
    rounded_tensor=tf.where(tensor>0,tf.math.floor(tensor),tf.math.ceil(tensor))
    mask_zero = tf.equal(rounded_tensor, 0)
    epsilon = 1e-7

    # Compute weight_log using log2
    weight_log = tf.math.log(tf.math.abs(tensor)+epsilon) / tf.math.log(2.0)
    print(weight_log)
    weight_log=tf.where(weight_log<0,tf.math.ceil(weight_log),tf.math.floor(weight_log))
    bit_pot = -2 ** bitwidth
    mask_condition = weight_log >= bit_pot
    
    # Initialize the result tensor with zeros
    result = tf.zeros_like(tensor, dtype=tf.float32)
    
    # Apply the condition where weight_log is valid
    result = tf.where(mask_condition, tf.where(tensor > 0, tf.math.abs(weight_log), weight_log), result)
    
    return result



In [44]:
lq=LQ(c,4)
lq


tf.Tensor(
[-9.4266586e+00 -4.2853994e+00 -5.7004318e+00 -3.5484347e+00
 -1.2265083e+00 -1.1561189e+00  1.7198263e-07 -2.4109323e+00
 -2.2265079e+00], shape=(9,), dtype=float32)


<tf.Tensor: shape=(9,), dtype=float32, numpy=array([ 9., -4.,  5.,  3.,  1., -1.,  0., -2.,  2.], dtype=float32)>

In [38]:
tf.math.floor(-4.2854023)

<tf.Tensor: shape=(), dtype=float32, numpy=-5.0>